In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import joblib

In [33]:
df=pd.read_csv('CreditScore_test.csv')

In [34]:
df.shape

(20000, 305)

In [35]:
df.isna().sum().head()

x001       0
x002    4247
x003    4247
x004    4243
x005    1243
dtype: int64

In [60]:
def preprocess(df):
    df.drop(['x242','x295','x304','x098','x155','x259','x255','x257','x256'],axis=1,inplace=True)
    df.drop(['x094','x067','x096','x095'],axis=1,inplace=True)
    null_clms=df.loc[:,df.isna().any()]
    df=df.fillna(null_clms.median())
    return df

In [61]:
df=preprocess(df)

In [63]:
x_final=df.drop('y',axis=1)
y_final=df.y

In [64]:
scale=joblib.load('scale.pkl')

In [74]:
x_final=scale.transform(x_final)

## XGBoost

In [69]:
xg=joblib.load('xgboost.pkl')

[17:20:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [72]:
pre=xg.predict(x_final)

In [71]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [73]:
print(r2_score(y_final,pre))
print(mean_absolute_error(y_final,pre))
print(mean_squared_error(y_final,pre))
print(np.sqrt(mean_squared_error(y_final,pre)))

0.9440373166987136
20.621705856323242
790.8436667187715
28.121942797729524


In [75]:
y=pd.Series(y_final,name='Actual')

In [76]:
y_pre=pd.Series(pre,name='Predicted')

In [77]:
Result=pd.concat([y,y_pre],axis=1)

In [79]:
Result.head()

,Actual,Predicted
0,406,386.843872
1,679,723.666382
2,750,766.015137
3,515,496.365814
4,589,530.476868


In [82]:
Result.to_csv('Result.csv',index=False)